<a href="https://colab.research.google.com/github/Vikas-Yadav-79/Intern-Task/blob/IMD/LSTM_GRU_21BCP379.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

df = pd.read_csv('/content/bbc (1).csv')
print(df.head())


   Unnamed: 0                                            Article     Class
0           0  Ad sales boost Time Warner profit\n\nQuarterly...  business
1           1  Dollar gains on Greenspan speech\n\nThe dollar...  business
2           2  Yukos unit buyer faces loan claim\n\nThe owner...  business
3           3  High fuel prices hit BA's profits\n\nBritish A...  business
4           4  Pernod takeover talk lifts Domecq\n\nShares in...  business


In [ ]:
import nltk
from nltk.corpus import stopwords
import string

nltk.download('stopwords')

def preprocess_text(text):
    stop_words = set(stopwords.words('english'))
    text = text.lower()  # Convert to lowercase
    text = ''.join([c for c in text if c not in string.punctuation])  # Remove punctuation
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stopwords
    return text

df['Article'] = df['Article'].apply(preprocess_text)
print(df['Article'].head())


In [4]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['Article'])

sequences = tokenizer.texts_to_sequences(df['Article'])

word_index = tokenizer.word_index
print(f"Found {len(word_index)} unique tokens.")


Found 32069 unique tokens.


In [5]:
from keras.preprocessing.sequence import pad_sequences

max_len = max(len(seq) for seq in sequences)

data = pad_sequences(sequences, maxlen=max_len)
print(data.shape)


(1912, 2237)


In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(df['Class'])
labels = to_categorical(labels)  # Convert to one-hot encoded format

X_train, X_temp, y_train, y_temp = train_test_split(data, labels, test_size=0.3, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=(2/3), random_state=1)

print(f"Training data shape: {X_train.shape}, Training labels shape: {y_train.shape}")
print(f"Validation data shape: {X_val.shape}, Validation labels shape: {y_val.shape}")
print(f"Test data shape: {X_test.shape}, Test labels shape: {y_test.shape}")


Training data shape: (1338, 2237), Training labels shape: (1338, 5)
Validation data shape: (191, 2237), Validation labels shape: (191, 5)
Test data shape: (383, 2237), Test labels shape: (383, 5)


In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.optimizers import Adam

embedding_dim = 100  # Size of word embeddings

def create_lstm_model():
    model = Sequential()
    model.add(Embedding(len(word_index) + 1, embedding_dim, input_length=max_len))
    model.add(LSTM(128))
    model.add(Dense(labels.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

lstm_model = create_lstm_model()
lstm_model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val), batch_size=32)


Epoch 1/5
42/42 [==============================] - 14s 266ms/step - loss: 1.5648 - accuracy: 0.3169 - val_loss: 1.5293 - val_accuracy: 0.5445
Epoch 2/5
42/42 [==============================] - 11s 263ms/step - loss: 1.1601 - accuracy: 0.8154 - val_loss: 0.7819 - val_accuracy: 0.7120
Epoch 3/5
42/42 [==============================] - 10s 249ms/step - loss: 0.3501 - accuracy: 0.9148 - val_loss: 0.3294 - val_accuracy: 0.8953
Epoch 4/5
42/42 [==============================] - 13s 302ms/step - loss: 0.3076 - accuracy: 0.9477 - val_loss: 0.4766 - val_accuracy: 0.8429
Epoch 5/5
42/42 [==============================] - 9s 206ms/step - loss: 0.2930 - accuracy: 0.9141 - val_loss: 0.6231 - val_accuracy: 0.7749


In [8]:
from tensorflow.keras.layers import GRU

def create_gru_model():
    model = Sequential()
    model.add(Embedding(len(word_index) + 1, embedding_dim, input_length=max_len))
    model.add(GRU(128))
    model.add(Dense(labels.shape[1], activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    return model

gru_model = create_gru_model()
gru_model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val), batch_size=32)


Epoch 1/5
42/42 [==============================] - 13s 266ms/step - loss: 1.5700 - accuracy: 0.2735 - val_loss: 1.5380 - val_accuracy: 0.3037
Epoch 2/5
42/42 [==============================] - 10s 240ms/step - loss: 1.1959 - accuracy: 0.5306 - val_loss: 1.1074 - val_accuracy: 0.5759
Epoch 3/5
42/42 [==============================] - 8s 190ms/step - loss: 0.4110 - accuracy: 0.9081 - val_loss: 0.6663 - val_accuracy: 0.7906
Epoch 4/5
42/42 [==============================] - 10s 240ms/step - loss: 0.0337 - accuracy: 0.9948 - val_loss: 0.6508 - val_accuracy: 0.7696
Epoch 5/5
42/42 [==============================] - 9s 223ms/step - loss: 0.0064 - accuracy: 0.9993 - val_loss: 0.5470 - val_accuracy: 0.8272


In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
import numpy as np

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_test, axis=1)

    accuracy = accuracy_score(y_true, y_pred_classes)
    precision = precision_score(y_true, y_pred_classes, average='weighted')
    recall = recall_score(y_true, y_pred_classes, average='weighted')

    return accuracy, precision, recall

lstm_accuracy, lstm_precision, lstm_recall = evaluate_model(lstm_model, X_test, y_test)
lstm_params = lstm_model.count_params()

print(f"LSTM Model Accuracy: {lstm_accuracy}")
print(f"LSTM Model Precision: {lstm_precision}")
print(f"LSTM Model Recall: {lstm_recall}")
print(f"LSTM Model Parameters: {lstm_params}")


12/12 [==============================] - 1s 37ms/step
LSTM Model Accuracy: 0.793733681462141
LSTM Model Precision: 0.8049712368362776
LSTM Model Recall: 0.793733681462141
LSTM Model Parameters: 3324893


In [10]:
gru_accuracy, gru_precision, gru_recall = evaluate_model(gru_model, X_test, y_test)
gru_params = gru_model.count_params()

print(f"GRU Model Accuracy: {gru_accuracy}")
print(f"GRU Model Precision: {gru_precision}")
print(f"GRU Model Recall: {gru_recall}")
print(f"GRU Model Parameters: {gru_params}")


12/12 [==============================] - 1s 40ms/step
GRU Model Accuracy: 0.814621409921671
GRU Model Precision: 0.8186818651008703
GRU Model Recall: 0.814621409921671
GRU Model Parameters: 3295965


In [11]:
from tensorflow.keras.layers import Activation

activation_functions = ['relu', 'tanh', 'sigmoid', 'elu', 'softmax']

results = []

for activation in activation_functions:
    lstm_model = Sequential()
    lstm_model.add(Embedding(len(word_index) + 1, embedding_dim, input_length=max_len))
    lstm_model.add(LSTM(128))
    lstm_model.add(Dense(labels.shape[1]))
    lstm_model.add(Activation(activation))
    lstm_model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    lstm_model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val), batch_size=32)

    lstm_accuracy, lstm_precision, lstm_recall = evaluate_model(lstm_model, X_test, y_test)
    results.append((f'LSTM with {activation}', lstm_accuracy, lstm_precision, lstm_recall))

    gru_model = Sequential()
    gru_model.add(Embedding(len(word_index) + 1, embedding_dim, input_length=max_len))
    gru_model.add(GRU(128))
    gru_model.add(Dense(labels.shape[1]))
    gru_model.add(Activation(activation))
    gru_model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    gru_model.fit(X_train, y_train, epochs=5, validation_data=(X_val, y_val), batch_size=32)

    gru_accuracy, gru_precision, gru_recall = evaluate_model(gru_model, X_test, y_test)
    results.append((f'GRU with {activation}', gru_accuracy, gru_precision, gru_recall))

with open('activation_results.txt', 'w') as f:
    for result in results:
        f.write(f"{result[0]}:\nAccuracy: {result[1]}\nPrecision: {result[2]}\nRecall: {result[3]}\n\n")


Epoch 1/5
42/42 [==============================] - 15s 245ms/step - loss: nan - accuracy: 0.2765 - val_loss: nan - val_accuracy: 0.2513
Epoch 2/5
42/42 [==============================] - 10s 227ms/step - loss: nan - accuracy: 0.2735 - val_loss: nan - val_accuracy: 0.2513
Epoch 3/5
42/42 [==============================] - 8s 201ms/step - loss: nan - accuracy: 0.2735 - val_loss: nan - val_accuracy: 0.2513
Epoch 4/5
42/42 [==============================] - 8s 189ms/step - loss: nan - accuracy: 0.2735 - val_loss: nan - val_accuracy: 0.2513
Epoch 5/5
12/12 [==============================] - 1s 38ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
42/42 [==============================] - 12s 231ms/step - loss: nan - accuracy: 0.2676 - val_loss: nan - val_accuracy: 0.2513
Epoch 2/5
42/42 [==============================] - 7s 174ms/step - loss: nan - accuracy: 0.2735 - val_loss: nan - val_accuracy: 0.2513
Epoch 3/5
42/42 [==============================] - 9s 209ms/step - loss: nan - accuracy: 0.2735 - val_loss: nan - val_accuracy: 0.2513
Epoch 4/5
42/42 [==============================] - 7s 163ms/step - loss: nan - accuracy: 0.2735 - val_loss: nan - val_accuracy: 0.2513
Epoch 5/5
12/12 [==============================] - 1s 36ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
42/42 [==============================] - 12s 210ms/step - loss: 8.6499 - accuracy: 0.2010 - val_loss: 7.8899 - val_accuracy: 0.2147
Epoch 2/5
42/42 [==============================] - 9s 208ms/step - loss: 8.4001 - accuracy: 0.2392 - val_loss: 10.0422 - val_accuracy: 0.2251
Epoch 3/5
42/42 [==============================] - 8s 202ms/step - loss: 8.9053 - accuracy: 0.2795 - val_loss: 10.1266 - val_accuracy: 0.2513
Epoch 4/5
42/42 [==============================] - 7s 163ms/step - loss: 10.9984 - accuracy: 0.2735 - val_loss: 10.1266 - val_accuracy: 0.2513
Epoch 5/5
12/12 [==============================] - 1s 26ms/step


...

Epoch 1/5
42/42 [==============================] - 11s 213ms/step - loss: 1.5035 - accuracy: 0.3132 - val_loss: 1.2231 - val_accuracy: 0.4607
Epoch 2/5
42/42 [==============================] - 7s 170ms/step - loss: 0.9477 - accuracy: 0.6510 - val_loss: 0.9985 - val_accuracy: 0.5550
Epoch 3/5
42/42 [==============================] - 7s 172ms/step - loss: 0.8143 - accuracy: 0.7309 - val_loss: 0.7680 - val_accuracy: 0.7539
Epoch 4/5
42/42 [==============================] - 8s 186ms/step - loss: 0.2810 - accuracy: 0.9327 - val_loss: 0.5170 - val_accuracy: 0.8220
Epoch 5/5
12/12 [==============================] - 1s 39ms/step
Epoch 1/5
42/42 [==============================] - 11s 207ms/step - loss: 1.5641 - accuracy: 0.2870 - val_loss: 1.5359 - val_accuracy: 0.2513
Epoch 2/5
42/42 [==============================] - 8s 185ms/step - loss: 1.1664 - accuracy: 0.5172 - val_loss: 0.9538 - val_accuracy: 0.6597
Epoch 3/5
42/42 [==============================] - 6s 154ms/step - loss: 0.2549 - accura

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
42/42 [==============================] - 9s 173ms/step - loss: 7.6199 - accuracy: 0.2205 - val_loss: 10.0424 - val_accuracy: 0.2618
Epoch 2/5
42/42 [==============================] - 8s 185ms/step - loss: 9.3026 - accuracy: 0.2220 - val_loss: 7.8092 - val_accuracy: 0.2408
Epoch 3/5
42/42 [==============================] - 7s 160ms/step - loss: 8.0430 - accuracy: 0.2145 - val_loss: 9.7890 - val_accuracy: 0.2356
Epoch 4/5
42/42 [==============================] - 8s 179ms/step - loss: 9.5652 - accuracy: 0.2332 - val_loss: 9.4514 - val_accuracy: 0.2565
Epoch 5/5
12/12 [==============================] - 1s 35ms/step


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch 1/5
42/42 [==============================] - 11s 215ms/step - loss: 1.5591 - accuracy: 0.2810 - val_loss: 1.5976 - val_accuracy: 0.2513
Epoch 2/5
42/42 [==============================] - 7s 173ms/step - loss: 0.9162 - accuracy: 0.6390 - val_loss: 0.5554 - val_accuracy: 0.8534
Epoch 3/5
42/42 [==============================] - 7s 175ms/step - loss: 0.3286 - accuracy: 0.9312 - val_loss: 0.5020 - val_accuracy: 0.8325
Epoch 4/5
42/42 [==============================] - 7s 165ms/step - loss: 0.0967 - accuracy: 0.9836 - val_loss: 0.4138 - val_accuracy: 0.8639
Epoch 5/5
12/12 [==============================] - 1s 36ms/step
Epoch 1/5
42/42 [==============================] - 10s 180ms/step - loss: 1.5684 - accuracy: 0.2623 - val_loss: 1.5204 - val_accuracy: 0.2880
Epoch 2/5
42/42 [==============================] - 7s 174ms/step - loss: 1.3334 - accuracy: 0.6510 - val_loss: 1.1854 - val_accuracy: 0.6335
Epoch 3/5
42/42 [==============================] - 6s 151ms/step - loss: 0.4558 - accura

In [13]:
from sklearn.model_selection import KFold

def cross_validate_model(model_creator, X, y, n_splits=3):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)
    accuracies = []
    class_accuracies = []

    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = model_creator()
        model.fit(X_train, y_train, epochs=5, batch_size=32)

        y_pred = model.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true = np.argmax(y_test, axis=1)

        accuracy = accuracy_score(y_true, y_pred_classes)
        class_accuracy = [accuracy_score(y_true == i, y_pred_classes == i) for i in range(labels.shape[1])]

        accuracies.append(accuracy)
        class_accuracies.append(class_accuracy)

    return accuracies, class_accuracies


lstm_accuracies, lstm_class_accuracies = cross_validate_model(create_lstm_model, data, labels)
gru_accuracies, gru_class_accuracies = cross_validate_model(create_gru_model, data, labels)

# Save the results to a file
with open('cross_validation_results.txt', 'w') as f:
    f.write(f"LSTM Model Cross-Validation:\nAccuracies: {lstm_accuracies}\nClass-wise Accuracies: {lstm_class_accuracies}\n\n")
    f.write(f"GRU Model Cross-Validation:\nAccuracies: {gru_accuracies}\nClass-wise Accuracies: {gru_class_accuracies}\n")


Epoch 1/5
40/40 [==============================] - 10s 180ms/step - loss: 1.5154 - accuracy: 0.2991
Epoch 2/5
40/40 [==============================] - 7s 176ms/step - loss: 1.0631 - accuracy: 0.5777
Epoch 3/5
40/40 [==============================] - 7s 169ms/step - loss: 0.5907 - accuracy: 0.8265
Epoch 4/5
40/40 [==============================] - 7s 169ms/step - loss: 0.2158 - accuracy: 0.9403
Epoch 5/5
20/20 [==============================] - 1s 33ms/step
Epoch 1/5
40/40 [==============================] - 10s 197ms/step - loss: 1.5684 - accuracy: 0.3075
Epoch 2/5
40/40 [==============================] - 6s 159ms/step - loss: 1.1053 - accuracy: 0.6573
Epoch 3/5
40/40 [==============================] - 7s 171ms/step - loss: 0.3037 - accuracy: 0.9224
Epoch 4/5
40/40 [==============================] - 6s 145ms/step - loss: 0.1556 - accuracy: 0.9788
Epoch 5/5
20/20 [==============================] - 1s 39ms/step
Epoch 1/5
40/40 [==============================] - 10s 192ms/step - loss: 1.56

In [14]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

def cross_validate_model(model_creator, X, y, n_splits=3):
    kfold = KFold(n_splits=n_splits, shuffle=True, random_state=1)
    accuracies = []
    class_accuracies = []

    for fold_idx, (train_index, test_index) in enumerate(kfold.split(X)):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model = model_creator()
        model.fit(X_train, y_train, epochs=5, batch_size=32)

        y_pred = model.predict(X_test)
        y_pred_classes = np.argmax(y_pred, axis=1)
        y_true = np.argmax(y_test, axis=1)

        accuracy = accuracy_score(y_true, y_pred_classes)
        accuracies.append(accuracy)

        class_accuracy = []
        for class_idx in range(y.shape[1]):
            class_accuracy.append(accuracy_score(y_true[y_true == class_idx], y_pred_classes[y_true == class_idx]))
        class_accuracies.append(class_accuracy)

        print(f"Fold {fold_idx+1} - Overall Accuracy: {accuracy}, Class-wise Accuracies: {class_accuracy}")

    return accuracies, class_accuracies

# Perform cross-validation for LSTM model
lstm_accuracies, lstm_class_accuracies = cross_validate_model(create_lstm_model, data, labels)

# Perform cross-validation for GRU model
gru_accuracies, gru_class_accuracies = cross_validate_model(create_gru_model, data, labels)


Epoch 1/5
40/40 [==============================] - 11s 163ms/step - loss: 1.5356 - accuracy: 0.3046
Epoch 2/5
40/40 [==============================] - 7s 188ms/step - loss: 1.0577 - accuracy: 0.6429
Epoch 3/5
40/40 [==============================] - 5s 135ms/step - loss: 0.4596 - accuracy: 0.8234
Epoch 4/5
40/40 [==============================] - 7s 173ms/step - loss: 0.2713 - accuracy: 0.9270
Epoch 5/5
20/20 [==============================] - 2s 50ms/step
Fold 1 - Overall Accuracy: 0.9090909090909091, Class-wise Accuracies: [0.8950617283950617, 0.9402985074626866, 0.9793103448275862, 0.8840579710144928, 0.828125]
Epoch 1/5
40/40 [==============================] - 12s 229ms/step - loss: 1.5475 - accuracy: 0.2949
Epoch 2/5
40/40 [==============================] - 6s 147ms/step - loss: 1.0443 - accuracy: 0.6463
Epoch 3/5
40/40 [==============================] - 7s 178ms/step - loss: 0.7016 - accuracy: 0.7427
Epoch 4/5
40/40 [==============================] - 6s 149ms/step - loss: 0.3908 

In [15]:
def write_results_to_file(filename, model_name, accuracies, class_accuracies):
    with open(filename, 'a') as f:
        f.write(f"{model_name} Results:\n")
        f.write("Fold\tOverall Accuracy\tClass-wise Accuracies\n")
        for i in range(len(accuracies)):
            f.write(f"{i+1}\t{accuracies[i]}\t{class_accuracies[i]}\n")
        f.write("\n")

# Perform cross-validation for LSTM model
lstm_accuracies, lstm_class_accuracies = cross_validate_model(create_lstm_model, data, labels)

# Write LSTM results to a file
write_results_to_file("cross_validation_results_1.txt", "LSTM", lstm_accuracies, lstm_class_accuracies)

# Perform cross-validation for GRU model
gru_accuracies, gru_class_accuracies = cross_validate_model(create_gru_model, data, labels)

# Write GRU results to a file
write_results_to_file("cross_validation_results_1.txt", "GRU", gru_accuracies, gru_class_accuracies)


Epoch 1/5
40/40 [==============================] - 10s 168ms/step - loss: 1.5218 - accuracy: 0.2881
Epoch 2/5
40/40 [==============================] - 7s 170ms/step - loss: 1.2492 - accuracy: 0.4859
Epoch 3/5
40/40 [==============================] - 6s 153ms/step - loss: 0.8965 - accuracy: 0.7339
Epoch 4/5
40/40 [==============================] - 6s 156ms/step - loss: 0.3355 - accuracy: 0.9278
Epoch 5/5
20/20 [==============================] - 1s 33ms/step
Fold 1 - Overall Accuracy: 0.7931034482758621, Class-wise Accuracies: [0.9259259259259259, 0.9328358208955224, 0.31724137931034485, 0.8695652173913043, 0.9765625]
Epoch 1/5
40/40 [==============================] - 9s 157ms/step - loss: 1.5752 - accuracy: 0.3129
Epoch 2/5
40/40 [==============================] - 7s 182ms/step - loss: 1.0069 - accuracy: 0.6306
Epoch 3/5
40/40 [==============================] - 6s 155ms/step - loss: 0.4534 - accuracy: 0.8933
Epoch 4/5
40/40 [==============================] - 7s 166ms/step - loss: 0.1181